In [1]:
import pandas as pd

Loading lists of samples

In [2]:
androidListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\androidSamples.csv")
awsListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\awsSamples.csv")
azureListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\azureSamples.csv")
springListSamples = pd.read_csv("..\\..\\1-GettingQuestions\\samplesWithQuestions\\springSamples.csv")

In [7]:
def getSampleDataframe(name):
    name = name.replace("/", "\\")
    sample = pd.read_csv("..\\..\\2-ExtractingMetrics\\metrics\\"+name+".csv", parse_dates=True)
    sampleAverage = sample[sample["type"] == "average"]
    sampleAverage["numberJavaFiles"] = sample[sample["type"] == "absolute"]["numberJavaFiles"].values
    sampleAverage["CountDeclClass"] = sample[sample["type"] == "absolute"]["CountDeclClass"].values
    sampleAverage["readability"] = (sampleAverage["readability"] * sampleAverage["numberJavaFiles"]) / sampleAverage["CountDeclClass"]
    sampleAverage['commitDate'] = pd.to_datetime(sampleAverage['commitDate'].astype(str).str[:-6])
    sampleAverage.set_index("commitDate", inplace=True)
    sampleAverage.dropna(axis=1, how="all", inplace=True)
    sampleAverage = sampleAverage.groupby([(sampleAverage.index.year), (sampleAverage.index.month)]).mean()
    generic = pd.DataFrame(index=pd.date_range(start="2013-04-15", end="2020-09-30", freq="M", name="commitDate"))
    generic = generic.groupby([(generic.index.year), (generic.index.month)]).sum()
    return pd.concat([generic, sampleAverage], axis=1).fillna(method='ffill').dropna(how="all")

For each framework, load metrics per samples and group by month

In [8]:
androidSamples = pd.concat([getSampleDataframe(sample) for sample in androidListSamples["path"]], axis=0)
awsSamples = pd.concat([getSampleDataframe(sample) for sample in awsListSamples["path"]], axis=0)
azureSamples = pd.concat([getSampleDataframe(sample) for sample in azureListSamples["path"]], axis=0)
springSamples = pd.concat([getSampleDataframe(sample) for sample in springListSamples["path"]], axis=0)

<ipython-input-7-153a14f4a358>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleAverage["numberJavaFiles"] = sample[sample["type"] == "absolute"]["numberJavaFiles"].values
<ipython-input-7-153a14f4a358>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleAverage["CountDeclClass"] = sample[sample["type"] == "absolute"]["CountDeclClass"].values
<ipython-input-7-153a14f4a358>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [9]:
androidSamples = androidSamples.groupby(level=[0,1]).mean()
awsSamples = awsSamples.groupby(level=[0,1]).mean()
azureSamples = azureSamples.groupby(level=[0,1]).mean()
springSamples = springSamples.groupby(level=[0,1]).mean()

Join all framework in one DataFrame

In [18]:
allMetricsAverage = pd.concat([androidSamples, awsSamples, azureSamples, springSamples], axis=0)

In [19]:
allMetrics = allMetricsAverage

In [20]:
allMetrics = allMetrics.groupby(level=[0,1]).mean()

In [21]:
def generateStats(dataframe):
    stats = pd.DataFrame(index = dataframe.min().index)
    stats["min"] = dataframe.min()
    stats["max"] = dataframe.max()
    stats["avg"] = dataframe.mean()
    stats["med"] = dataframe.median()
    stats["std"] = dataframe.std()
    stats.to_csv("stats.csv")

Obtaining stats

In [22]:
generateStats(allMetrics)

Loading questions

In [287]:
androidQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\androidQuestions.csv")
awsQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\awsQuestions.csv")
azureQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\azureQuestions.csv")
springQuestions = pd.read_csv("..\\..\\1-GettingQuestions\\questions\\springQuestions.csv")

In [288]:
allQuestions = pd.concat([androidQuestions, awsQuestions, azureQuestions, springQuestions], axis=0, ignore_index=True)

In [ ]:
allQuestions['creationDate'] = pd.to_datetime(allQuestions['creationDate'])
allQuestions.sort_values(by="creationDate", inplace=True)
allQuestions.set_index(allQuestions['creationDate'], inplace=True)
allQuestions["questions"] = 1
del allQuestions["ownerUserId"]
del allQuestions["postTypeId"]
del allQuestions["acceptedAnswerId"]
del allQuestions["id"]
del allQuestions['creationDate']
allQuestions=allQuestions.groupby([(allQuestions.index.year), (allQuestions.index.month)]).sum()

In [290]:
allQuestions["questions"]=allQuestions.cumsum()

Merging metrics and questions

In [292]:
metricsAndQuestions = pd.concat([allMetrics, allQuestions], axis=1)
metricsAndQuestions.fillna(method="ffill", inplace=True)
metricsAndQuestions.fillna(0, inplace=True)

In [293]:
metricsAndQuestions = metricsAndQuestions.drop((2020, 9))

Saving

In [294]:
metricsAndQuestions.to_csv("metricsAndQuestions.csv")